# Tango Green Mode Comparison: Asyncio vs. Gevent
This notebook demonstrates the non-blocking nature of Tango devices configured with Asyncio and Gevent green modes.



## 1. Setup and Imports

Run the servers:
- python gevent_device_server.py test -nodb -dlist test/gevent/1 --port 10000
- python asyncio_device_server.py test -nodb -dlist test/asyncio/1 --port 10001

In [14]:
import time
import concurrent.futures
from tango import DeviceProxy, DevState

# Update these strings with your actual device registered names
# ASYNC_DEVICE_NAME = 'test/asyncio/1'
# GEVENT_DEVICE_NAME = 'test/gevent/1'

GEVENT_DEVICE_NAME = "tango://localhost:10000/test/gevent/1#dbase=no"
ASYNC_DEVICE_NAME = "tango://localhost:10001/test/asyncio/1#dbase=no"

dev_async = DeviceProxy(ASYNC_DEVICE_NAME)
dev_gevent = DeviceProxy(GEVENT_DEVICE_NAME)

def monitor_state(device, duration=5):
    """Polls the device state for a few seconds to show changes."""
    start = time.time()
    print(f"Monitoring {device.name()}...")
    while time.time() - start < duration:
        print(f"Time: {time.time()-start:.1f}s | State: {device.state()}", end='\r')
        time.sleep(0.5)
    print("\nMonitoring finished.")

## 2. Testing Server 1: Asyncio
In Asyncio mode, the long_running_command uses await asyncio.sleep(2). This allows other requests (like reading the state or attributes) to pass through while the sleep is happening.

In [15]:
print("--- Testing Asyncio Device ---")

# We run the long command in a separate thread/future so we can 
# immediately try to read other things in this notebook.
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 1. Trigger the long-running command (takes 2 seconds)
    executor.submit(dev_async.long_running_command)
    
    # 2. Immediately try to read an attribute
    # If concurrency works, this should return even while long_running_command is sleeping.
    print(f"Immediate Attribute Read: {dev_async.test_attribute}")
    
    # 3. Monitor the state transitions (OPEN -> CLOSE)
    monitor_state(dev_async, duration=3)

--- Testing Asyncio Device ---
Immediate Attribute Read: 42.0
Monitoring test/asyncio/1...
Time: 2.5s | State: CLOSE
Monitoring finished.


## 3. Testing Server 2: Gevent
In Gevent mode, the server uses gevent.sleep(2). This implicitly yields the "Greenlet" hub, allowing the client to read attributes concurrently without being blocked by the "Monitor Lock."

In [16]:
print("--- Testing Gevent Device ---")

with concurrent.futures.ThreadPoolExecutor() as executor:
    # 1. Trigger the background task (sets state to INSERT for 15s)
    dev_gevent.background_task_command()
    print("Background task triggered on Gevent device.")
    
    # 2. Trigger the long-running command (sets state to OPEN for 2s)
    executor.submit(dev_gevent.long_running_command)
    
    # 3. Read attribute (should not be blocked by the 2s sleep)
    print(f"Attribute read during Gevent sleep: {dev_gevent.test_attribute}")

    # 4. Monitor state transitions
    # You should see INSERT (from background) then OPEN/CLOSE (from command)
    monitor_state(dev_gevent, duration=5)

--- Testing Gevent Device ---
Background task triggered on Gevent device.
Attribute read during Gevent sleep: 42.0
Monitoring test/gevent/1...
Time: 4.5s | State: CLOSE
Monitoring finished.


## 4.Concurrency Benchmark Code


In [18]:
import time
import concurrent.futures

def benchmark_device(device_proxy, label):
    print(f"--- Benchmarking {label} ({device_proxy.name()}) ---")
    # 1. Increase timeout to 10 seconds to account for network/CORBA overhead
    device_proxy.set_timeout_millis(10000)
    # We will fire 10 requests at once
    num_requests = 10
    start_time = time.time()

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_requests) as executor:
        # We call 'test_attribute' 10 times. 
        # Inside the server, each call triggers a 2-second sleep.
        futures = [executor.submit(lambda: device_proxy.test_attribute) for _ in range(num_requests)]
        results = [f.result() for f in futures]

    end_time = time.time()
    total_duration = end_time - start_time
    
    print(f"Results received: {results}")
    print(f"Total time for {num_requests} concurrent reads: {total_duration:.2f} seconds")
    
    if total_duration < 3.0:
        print(f"✅ SUCCESS: {label} is running CONCURRENTLY.")
    else:
        print(f"❌ FAILURE: {label} is SERIALIZING requests (Blocking).")
    print("\n")

# Run the benchmarks
benchmark_device(dev_async, "Asyncio Server")
benchmark_device(dev_gevent, "Gevent Server")

--- Benchmarking Asyncio Server (test/asyncio/1) ---


DevFailed: DevFailed[
    DevError[
        desc = TRANSIENT CORBA system exception: TRANSIENT_CallTimedout
        origin = DeviceProxy:read_attribute
        reason = API_CorbaException
        severity = ERR
    ],
    DevError[
        desc = Timeout (10000 mS) exceeded on device test/asyncio/1
        origin = DeviceProxy:read_attribute
        reason = API_DeviceTimedOut
        severity = ERR
    ]
]

Test Case,Expected Result,Why?
Attribute Read,Returns quickly (~2s),"The green mode yields control during the sleep, so the attribute read request isn't queued behind the command."
State Polling,Shows intermediate states,"The device doesn't ""freeze"" while executing the command logic."
Background Task,State stays in INSERT,"In both servers, the ""background"" command returns immediately to the client, while the device continues working in the background."